## Intégrer Refashion depuis l'api pointsapport :

- Récupérer les données-eo-refashion depuis l'api pointsapport.
- Créer et mapper les données vers les tables Acteurs, Proposition de Services et Sous-catégories.
- Enregistrer chaque table dans un fichier CSV.

In [94]:
from sqlalchemy import create_engine
user = 
password = 
host = 
port = '33517'  # default PostgreSQL port is 5432
db_name = 'quefairedem_2657'

# Create the connection URL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'

# Create the engine
engine = create_engine(connection_string)

## Get data from point apport 

In [ ]:
import requests
import pandas as pd


def fetch_all_data(url):
    all_data = []
    while url:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data['results'])
            # Check if there's a next page link
            url = data.get('next', None)
            print(url)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break
    return all_data

api_url = "https://data.pointsapport.ademe.fr/data-fair/api/v1/datasets/donnees-eo-refashion/lines?size=10000"

data = fetch_all_data(api_url)

df = pd.DataFrame(data)



In [119]:
df_acteurtype = pd.read_sql_table('qfdmo_acteurtype', engine)
df_sources = pd.read_sql_table('qfdmo_source', engine)
df_da = pd.read_sql_table('qfdmo_displayedacteur', engine)
df_ps = pd.read_sql_table('qfdmo_propositionservice', engine)
df_ps['id'].max()
df_pssc = pd.read_sql_table('qfdmo_propositionservice_sous_categories', engine)
df_action = pd.read_sql_table('qfdmo_action', engine)
df_ac = pd.read_sql_table('qfdmo_acteur', engine)


/Users/hamzaa/data/quefairedemesobjets/venv/lib/python3.9/site-packages/pandas/io/sql.py:1665: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)
/Users/hamzaa/data/quefairedemesobjets/venv/lib/python3.9/site-packages/pandas/io/sql.py:1665: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [120]:
df_ac = pd.read_sql_table('qfdmo_acteur', engine)


/Users/hamzaa/data/quefairedemesobjets/venv/lib/python3.9/site-packages/pandas/io/sql.py:1665: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [125]:
df_da.columns

Index(['nom', 'description', 'identifiant_unique', 'adresse',
       'adresse_complement', 'code_postal', 'ville', 'url', 'email',
       'location', 'telephone', 'multi_base', 'nom_commercial', 'nom_officiel',
       'manuel', 'label_reparacteur', 'siret', 'identifiant_externe', 'statut',
       'naf_principal', 'commentaires', 'cree_le', 'modifie_le', 'horaires',
       'acteur_type_id', 'source_id'],
      dtype='object')

In [129]:
df_ac['manuel'].unique()

array([False,  True])

### Mappers

In [112]:
column_mapping = {
    'id_point_apport_ou_reparation': 'identifiant_externe',
    'adresse_complement': 'adresse_complement',
    'type_de_point_de_collecte': 'acteur_type_id',
    'telephone': 'telephone',
    'siret': 'siret',
    'uniquement_sur_rdv': '',
    'exclusivite_de_reprisereparation': '',
    'filiere': '',
    'public_accueilli': '',
    'produitsdechets_acceptes': '',
    'labels_etou_bonus': '',
    'reprise': '',
    'point_de_reparation': '',
    'ecoorganisme': 'source_id',
    'adresse_format_ban': 'adresse',
    'nom_de_lorganisme': 'nom',
    'enseigne_commerciale':'nom_commercial',
    '_updatedAt':'cree_le',
    'site_web': 'url',
    'email': 'email',
    'perimetre_dintervention': '',
    'longitudewgs84': 'location',  
    'latitudewgs84': 'location',  
    'horaires_douverture': 'horaires',
    'consignes_dacces': 'commentaires',
}
def transform_action_id(action_type):
    map_eo_to_lvao = {
        "Point d'Apport Volontaire Publique": "point d'apport volontaire public",
        "Déchèterie": "déchèterie",
        "Association, entreprise de l’économie sociale et solidaire (ESS)": "ess",
        "Magasin / Franchise, Enseigne commerciale / Distributeur / Point de vente": "commerce",
        "Artisan, commerce indépendant": "artisan",
        "Solution en ligne (site web, app. mobile)": "acteur digital"
    }
    lvao_name = map_eo_to_lvao.get(action_type, None)
    map_lvao_name_to_id = {
        "point d'apport volontaire public": 10,
        "déchèterie": 7,
        "ess": 1,
        "commerce": 4,
        "artisan": 3,
        "acteur digital": 5
    }
    
    return map_lvao_name_to_id.get(lvao_name, None)

# Print the dictionary for visual confirmation
print(column_mapping)

{'id_point_apport_ou_reparation': 'identifiant_externe', 'adresse_complement': 'adresse_complement', 'type_de_point_de_collecte': 'acteur_type_id', 'telephone': 'telephone', 'siret': 'siret', 'uniquement_sur_rdv': '', 'exclusivite_de_reprisereparation': '', 'filiere': '', 'public_accueilli': '', 'produitsdechets_acceptes': '', 'labels_etou_bonus': '', 'reprise': '', 'point_de_reparation': '', 'ecoorganisme': 'source_id', 'adresse_format_ban': 'adresse', 'nom_de_lorganisme': 'nom', 'enseigne_commerciale': 'nom_commercial', '_updatedAt': 'cree_le', 'site_web': 'url', 'email': 'email', 'perimetre_dintervention': '', 'longitudewgs84': 'location', 'latitudewgs84': 'location', 'horaires_douverture': 'horaires', 'consignes_dacces': 'commentaires'}


### Transformations

#### Create Actors

In [126]:
from shapely.geometry import Point
from shapely import wkb
import re
import hashlib


selected_columns = ['acteur_type_id', 'adresse', 'code_postal', 'ville', 'email', 'siret']

def generate_unique_id(row):
    unique_str = '-'.join([str(row[col]) for col in selected_columns])
    return hashlib.sha256(unique_str.encode()).hexdigest()
def transform_acteur_type_id(value):
    mapping_dict = {
        "Solution en ligne (site web, app. mobile)": "en ligne (web, mobile)",
        "Artisan, commerce indépendant": "artisan, commerce indépendant",
        "Magasin / Franchise, Enseigne commerciale / Distributeur / Point de vente": "commerce",
        "Point d'Apport Volontaire Publique": "point d'apport volontaire public",
        "Association, entreprise de l’économie sociale et solidaire (ESS)": "Association, entreprise de l'ESS",
        "Déchèterie": "déchèterie",
    }
    nom_affiche = mapping_dict.get(value)
    id_value = df_acteurtype.loc[df_acteurtype['nom_affiche'] == nom_affiche, 'id'].values[0] if any(df_acteurtype['nom_affiche'] == nom_affiche) else None
    return id_value



def transform_location(longitude, latitude):
    point = Point(longitude, latitude)
    
    transformed_location_binary = wkb.dumps(point)
    transformed_location_hex = transformed_location_binary.hex()

    return transformed_location_hex

def transform_ecoorganisme(value):
    
    id_value = df_sources.loc[df_sources['nom'].str.lower() == value.lower(), 'id'].values[0] if any(df_sources['nom'].str.lower() == value.lower()) else None
    return id_value

def extract_details(row):
    pattern = re.compile(r'\b(\d{5})\s+(.*)')
    
    address = None
    postal_code = None
    city = None
     # Alternatively, skip processing if adress_ban is not a string (e.g., if it's NaN)
    if pd.isnull(row['adresse_format_ban']):
        return pd.Series([None, None, None])

    # Ensure adress_ban is treated as a string
    adress_ban = str(row['adresse_format_ban'])
    
    # Search for the pattern
    match = pattern.search(adress_ban)
    if match:
        postal_code = match.group(1)
        city = match.group(2)
        address = adress_ban[:match.start()].strip()
    
    return pd.Series([address, postal_code, city])

# Apply the function and assign the result to new columns
for old_col, new_col in column_mapping.items():
    if new_col: 
        if old_col == 'type_de_point_de_collecte':
            df[new_col] = df[old_col].apply(transform_acteur_type_id)
        elif old_col in ('longitudewgs84', 'latitudewgs84'):
            df['location'] = df.apply(lambda row: transform_location(row['longitudewgs84'], row['latitudewgs84']), axis=1)
        elif old_col == 'ecoorganisme':
            df[new_col] = df[old_col].apply(transform_ecoorganisme)
        elif old_col == 'adresse_format_ban':
            df[['adresse', 'code_postal', 'ville']] = df.apply(extract_details, axis=1)
        else:
            df[new_col] = df[old_col]
df['label_reparacteur']=False
df['multi_base']=False


df['identifiant_unique'] = df.apply(generate_unique_id, axis=1)
            


In [137]:
df['manuel']=False
df['statut']='ACTIF'
df['modifie_le'] = df['cree_le']


#### Create Proposition de services

In [ ]:
rows_list = []

for index, row in df.iterrows():
    acteur_id = row['identifiant_unique']
    action_id = transform_action_id(row['type_de_point_de_collecte'])
    sous_categories = row['produitsdechets_acceptes']
    if row['point_dapport_de_service_reparation']:
        acteur_service_id = 17
    elif row['point_dapport_pour_reemploi']:
        acteur_service_id = 4
    elif row['point_de_reparation']:
        acteur_service_id = 15
    elif row['point_de_collecte_ou_de_reprise_des_dechets']:
        acteur_service_id = 18
    else:
        continue  # Skip rows that don't match any criteria
    
    rows_list.append({"acteur_service_id": acteur_service_id, "action_id": action_id, "acteur_id": acteur_id, "sous_categories":sous_categories})

df_pds = pd.DataFrame(rows_list)
df_pds.index = range(659225, 659225 + len(df_pds))

df_pds['id'] = df_pds.index


#### Create sous categories

In [ ]:
rows_list=[]
sous_categories = { "Vêtement" : 107,
                   "Linge" : 104,
                   "Chaussure":109}
for index, row in df_pds.iterrows():
    products = str(row["sous_categories"]).split("|")
    for product in products:
        if product.strip() in sous_categories:
            rows_list.append({
                'propositionservice_id': row['id'], 
                'souscategorieobjet_id': sous_categories[product.strip()]
            })

df_sous_categories = pd.DataFrame(rows_list, columns=['propositionservice_id', 'souscategorieobjet_id'])

df_sous_categories

In [105]:


df[['identifiant_unique','acteur_type_id',
'adresse',
    'code_postal', 'ville',
 'adresse_complement',
 'commentaires',
 'email',
 'horaires',
 'identifiant_externe',
 'label_reparacteur',
 'location',
 'nom_commercial',
 'nom_officiel',
 'siret',
 'source_id',
 'telephone',
 'url']].to_csv('refashion_actors.csv')


In [ ]:

df[['identifiant_unique','acteur_type_id',
'adresse',
    'code_postal', 'ville',
 'adresse_complement',
 'commentaires',
 'email',
 'horaires',
 'identifiant_externe',
 'label_reparacteur',
 'location',
 'nom_commercial',
 'nom',
    'cree_le',
    'modifie_le',
  'multi_base',
    'manuel',
    'statut',
 'siret',
 'source_id',
 'telephone',
 'url']].to_sql("qfdmo_acteur",engine, if_exists='append',index=False,method='multi',chunksize=1000)

In [91]:
df_pds[['acteur_service_id','action_id','acteur_id','id']].to_csv('refashion_propositionservice.csv')

In [93]:
df_sous_categories[['propositionservice_id','souscategorieobjet_id']].to_csv('refashion_sous_categories.csv')